## Excel dosyasındaki verinin yüklenmesini gerçekleştirme

In [1]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "MovieDatabase.csv"
loader = CSVLoader(filepath)

data = loader.load()

len(data)  # Check the number of elements loaded

c:\Users\Nihal\miniconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


9837

## Veriyi parçalarına ayırma(Chunking işlemi)

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_text(data)

print(f"Number of documents after splitting: {len(docs)}")  

docs[7]

TypeError: expected string or bytes-like object

## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


: 

In [ ]:
load_dotenv()  # Load environment variables from .env file

: 

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

: 

## ChromaDB üzerine kayıt işlemi

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

: 

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5 })

: 

In [ ]:
retrieved_docs = retriever.invoke("What is encoder?") #soruya göre dökümanları getir    

len(retrieved_docs)

print(retrieved_docs[5].page_content) # Get content of the 6th retrieved document   



: 

## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)

: 

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


: 

In [ ]:
system prompt = (
    "You are assistant for question-answering tasks."
    "Use the following context to answer the question at the end."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "\n\n"
    "{context}"
)



: 

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


: 

## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [ ]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)


: 

## RAG Zinciri Oluşturma (RAG + LLM)


In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

: 

## Kullanıcı sorgusunu çalıştırma

In [ ]:
response = rag_chain.invoke({"input": "Explain the transformer architecture?"})

print(response["answer"])

: 